In [62]:
library(tidyverse)
library(glue)
library(writexl)

In [70]:
beeswarm_tibble_d5 = qs::qread('beeswarm_da_results_d5.qs') %>%
mutate(time = 'Day5')

beeswarm_tibble_d14 = qs::qread('beeswarm_da_results_d14.qs') %>%
mutate(time = 'Day14')

beeswarm_tibble_all = bind_rows(beeswarm_tibble_d5, beeswarm_tibble_d14) %>%
relocate(time, .after='labels')

beeswarm_tibble_all %>% dim
beeswarm_tibble_all %>% head

[1] 8829   39

,logFC,logCPM,F,PValue,FDR,Nhood,SpatialFDR,polarity,labels,time,⋯,F_bl6,PValue_bl6,FDR_bl6,SpatialFDR_bl6,polarity_bl6,labels_fraction_bl6,n_cells_bl6,class,UMAP_1,UMAP_2
,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>,<chr>,<dbl>,<dbl>
1,4.012020,13.75433,42.58423,2.578259e-07,2.732955e-05,45,2.702642e-05,pos,a1_Tany__Astrocytes,Day5,⋯,0.2194957,0.642651494,0.80142422,0.79802141,none,1,145,other,-3.776624,1.7098639
2,4.496957,12.88463,34.39239,1.704485e-06,9.033771e-05,27,9.639028e-05,pos,a1_Tany__Astrocytes,Day5,⋯,0.4136810,0.589556305,0.77880033,0.77225981,none,1,65,other,-3.324144,0.5136201
3,-2.653233,13.69086,21.05188,6.756198e-05,2.387190e-03,39,2.336014e-03,neg,a1_Tany__Astrocytes,Day5,⋯,7.5613790,0.009793458,0.09337777,0.08601752,neg,1,189,other,-4.463944,1.0696673
4,2.620222,13.54501,19.18199,1.225320e-04,2.909334e-03,19,2.895304e-03,pos,a1_Tany__Astrocytes,Day5,⋯,0.2511149,0.619776319,0.79152156,0.78786039,none,1,129,other,-2.870708,-0.2249309
5,2.453920,14.08700,18.83448,1.372327e-04,2.909334e-03,44,2.895304e-03,pos,a1_Tany__Astrocytes,Day5,⋯,0.3916998,0.535917609,0.73775671,0.72828245,none,1,199,other,-5.377928,0.7945066
6,-2.632430,13.76906,16.64476,2.860984e-04,5.054405e-03,70,4.891354e-03,neg,a1_Tany__Astrocytes,Day5,⋯,10.3307140,0.003017245,0.06761332,0.06271712,neg,1,194,other,-4.416568,0.9841724


In [71]:
beeswarm_tibble = beeswarm_tibble_all %>%
group_by(labels, time, class) %>%
summarise(min_cells = min(n_cells),
          max_cells = max(n_cells),
          mean_cells = mean(n_cells),
          median_cells = median(n_cells),
          n_nhoods = n(),
          n_fgf1 = sum(polarity_fgf1 != 'none'),
          pos_fgf1 = sum(polarity_fgf1 == 'pos'),
          neg_fgf1 = sum(polarity_fgf1 == 'neg'),
          frac_fgf1 = n_fgf1/n_nhoods,
          frac_pos_fgf1 = pos_fgf1/n_fgf1,
          neg_pos_fgf1 = neg_fgf1/n_fgf1,
          n_bl6 = sum(polarity_bl6 != 'none'),
          pos_bl6 = sum(polarity_bl6 == 'pos'),
          neg_bl6 = sum(polarity_bl6 == 'neg'),
          frac_bl6 = n_bl6/n_nhoods,
          frac_pos_bl6 = pos_bl6/n_bl6,
          neg_pos_bl6 = neg_bl6/n_bl6,
          n_restored = sum(restored),
          pos_restored = sum(restored & polarity_fgf1 == 'pos'),
          neg_restored = sum(restored & polarity_fgf1 == 'neg'),
          frac_restored = n_restored/n_nhoods,
          frac_pos_restored_fgf1 = pos_restored/n_fgf1,
          frac_neg_restored_fgf1 = neg_restored/n_fgf1,
          frac_pos_restored_bl6 = pos_restored/n_bl6,
          frac_neg_restored_bl6 = neg_restored/n_bl6
          ) %>% 
ungroup %>%
mutate(class = case_when(class == 'neuron' ~ 'neuronal',
                         class == 'other' ~ 'non-neuronal'))


# Find the columns that contain "frac"
frac_cols <- grep("frac", names(beeswarm_tibble), value = TRUE)

# Multiply by 100 and round
beeswarm_tibble <- beeswarm_tibble %>%
  mutate(across(all_of(frac_cols), ~round(. * 100)))

`summarise()` has grouped output by 'labels', 'time'. You can override using
the `.groups` argument.


In [74]:
library(dplyr)
library(glue)

write_summary <- function(df) {
  glue("
  At day {df$time}, {df$class} cluster {df$labels} formed {df$n_nhoods} total neighborhoods, each with a median of {df$median_cells} cells. 
  Of these, {df$n_fgf1} ({df$frac_fgf1}%) experienced a shift in differential abundance (DA) due to FGF1 treatment in obob animals. 
  The obob response to FGF1 treatment was characterized by {df$pos_fgf1}/{df$n_fgf1} neighborhoods experiencing an increase in the abundance of cells from FGF1 treated animals compared to {df$neg_fgf1}/{df$n_fgf1} neighborhoods which saw a greater abundance of cells from obob animals which were treated with vehicle and pair fed. 
  Of the same {df$labels} neighborhoods, {df$n_bl6} saw DA shift due to transcriptional differences in genotype. 
  BL6 animals contributed more cells to {df$pos_bl6}/{df$n_bl6} affected neighborhoods, and {df$neg_bl6}/{df$n_bl6} neighborhoods were enriched for cells originating from vehicle treated pair-fed obob animals.
  FGF1 treatement in obob animals and the BL6 genotype were found to act in the same direction on {df$n_restored} neighborhoods. 
  FGF1 treatment and BL6 increased the abundance of cells from obob animals in {df$pos_restored} neighborhoods and decreased their abundance in {df$neg_restored} neighborhoods.")
}

# Now, apply the function to each row of the tibble
beeswarm_tibble = beeswarm_tibble %>%
  rowwise() %>%
  mutate(summary = write_summary(cur_data()))

beeswarm_tibble %>% head(1)
beeswarm_tibble %>% write_xlsx('beeswarm_tibble_summary.xlsx')

labels,time,class,min_cells,max_cells,mean_cells,median_cells,n_nhoods,n_fgf1,pos_fgf1,⋯,neg_pos_bl6,n_restored,pos_restored,neg_restored,frac_restored,frac_pos_restored_fgf1,frac_neg_restored_fgf1,frac_pos_restored_bl6,frac_neg_restored_bl6,summary
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<int>,<int>,<int>,⋯,<dbl>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<glue>
Agrp,Day14,neuronal,47,237,93.07692,88,143,12,5,⋯,0.8153846,7,0,7,5,0,58,0,5,"At day Day14, neuronal cluster Agrp formed 143 total neighborhoods, each with a median of 88 cells. Of these, 12 (8%) experienced a shift in differential abundance (DA) due to FGF1 treatment in obob animals. The obob response to FGF1 treatment was characterized by 5/12 neighborhoods experiencing an increase in the abundance of cells from FGF1 treated animals compared to 7/12 neighborhoods which saw a greater abundance of cells from obob animals which were treated with vehicle and pair fed. Of the same Agrp neighborhoods, 130 saw DA shift due to transcriptional differences in genotype. BL6 animals contributed more cells to 24/130 affected neighborhoods, and 106/130 neighborhoods were enriched for cells originating from vehicle treated pair-fed obob animals. FGF1 treatement in obob animals and the BL6 genotype were found to act in the same direction on 7 neighborhoods. FGF1 treatment and BL6 increased the abundance of cells from obob animals in 0 neighborhoods and decreased their abundance in 7 neighborhoods."


In [80]:
beeswarm_tibble %>% distinct(labels, class) %>% print(n=80)

# A tibble: 49 × 2
# Rowwise: 
   labels                     class       
   <chr>                      <chr>       
 1 Agrp                       neuronal    
 2 Astrocytes                 non-neuronal
 3 Epend                      non-neuronal
 4 Fibroblasts2__Fibroblasts3 non-neuronal
 5 Ghrh                       neuronal    
 6 Gm8773_Tac1__Hdc           neuronal    
 7 Gpr50                      neuronal    
 8 Htr3b                      neuronal    
 9 Kisspeptin                 neuronal    
10 MOL                        non-neuronal
11 Microglia                  non-neuronal
12 NFOL                       non-neuronal
13 NG2_OPC2                   non-neuronal
14 NG2_OPC2__Microglia        non-neuronal
15 Nfix_Htr2c                 neuronal    
16 Nr5a1_Bdnf                 neuronal    
17 Nr5a1_Bdnf__Nr5a1_Nfib     neuronal    
18 Nr5a1_Bdnf__Unassigned1    neuronal    
19 Nr5a1_Nfib                 neuronal    
20 Oxt__Unassigned1           neuronal    
21 Pomc_Glipr1__Slc17a6